In [1]:
import os
import re
import subprocess

import ipysheet

In [2]:
def generate_sheet(rows=5, columns=5):
    sheet = ipysheet.sheet(rows=rows, columns=columns, column_width=[20] + [5 for _ in range(1, columns)])
    for row in range(rows):
        for column in range(columns):
            ipysheet.cell(row, column, "")
    return sheet


In [3]:
sheet = generate_sheet(rows=13, columns=9)

ipysheet.cell(0, 0, "happened(wait, T)")
ipysheet.cell(1, 0, 'happened(select(policy_compliant(greeted_by("Bob", "Alice"))), T)')
ipysheet.cell(2, 0, 'happened(start(1), T)')
ipysheet.cell(3, 0, 'happened(stop(1), T)')
ipysheet.cell(4, 0, 'happened(move_through("Alice", "d12"), T)')
ipysheet.cell(5, 0, 'happened(move_through("Alice", "d23"), T)')
ipysheet.cell(6, 0, 'happened(move_through("Alice", "d34"), T)')
ipysheet.cell(7, 0, 'happened(greet("Alice", "Bob"), T)')
ipysheet.cell(8, 0, 'observed(in_room("Alice", "r2"), T)')
ipysheet.cell(9, 0, 'observed(in_room("Alice", "r3"), T)')
ipysheet.cell(10, 0, 'observed(in_room("Alice", "r4"), T)')
ipysheet.cell(11, 0, 'observed(greeted_by("Bob", "Alice"), T)')
ipysheet.cell(12, 0, '-observed(greeted_by("Bob", "Alice"), T)')

ipysheet.cell(0, 1, "X")
ipysheet.cell(0, 8, "X")
ipysheet.cell(1, 1, "X")
ipysheet.cell(2, 2, "X")
ipysheet.cell(3, 7, "X")
ipysheet.cell(4, 3, "X")
ipysheet.cell(5, 4, "X")
ipysheet.cell(6, 5, "X")
ipysheet.cell(7, 6, "X")

ipysheet.cell(8, 4, "X")
ipysheet.cell(9, 5, "X")
ipysheet.cell(10, 6, "X")

ipysheet.cell(11, 7, "X")
ipysheet.cell(12, 2, "X")
ipysheet.cell(12, 3, "X")
ipysheet.cell(12, 4, "X")
ipysheet.cell(12, 5, "X")
ipysheet.cell(12, 6, "X")

sheet

Sheet(cells=(Cell(column_end=0, column_start=0, row_end=0, row_start=0, type='text', value=''), Cell(column_en…

In [4]:
df = ipysheet.pandas_loader.to_dataframe(sheet)
df

,A,B,C,D,E,F,G,H,I
0,"happened(wait, T)",X,,,,,,,X
1,"happened(select(policy_compliant(greeted_by(""B...",X,,,,,,,
2,"happened(start(1), T)",,X,,,,,,
3,"happened(stop(1), T)",,,,,,,X,
4,"happened(move_through(""Alice"", ""d12""), T)",,,X,,,,,
5,"happened(move_through(""Alice"", ""d23""), T)",,,,X,,,,
6,"happened(move_through(""Alice"", ""d34""), T)",,,,,X,,,
7,"happened(greet(""Alice"", ""Bob""), T)",,,,,,X,,
8,"observed(in_room(""Alice"", ""r2""), T)",,,,X,,,,
9,"observed(in_room(""Alice"", ""r3""), T)",,,,,X,,,


In [5]:
fact_dict = {}

for rindex, row in df.iterrows():
    fact = None
    for cindex, cell in enumerate(row):
        if cindex == 0:
            fact = cell
            fact_dict[cell] = set()
        elif cell == "X":
            fact_dict[fact].add(cindex - 1)

fact_dict

{'happened(wait, T)': {0, 7},
 'happened(select(policy_compliant(greeted_by("Bob", "Alice"))), T)': {0},
 'happened(start(1), T)': {1},
 'happened(stop(1), T)': {6},
 'happened(move_through("Alice", "d12"), T)': {2},
 'happened(move_through("Alice", "d23"), T)': {3},
 'happened(move_through("Alice", "d34"), T)': {4},
 'happened(greet("Alice", "Bob"), T)': {5},
 'observed(in_room("Alice", "r2"), T)': {3},
 'observed(in_room("Alice", "r3"), T)': {4},
 'observed(in_room("Alice", "r4"), T)': {5},
 'observed(greeted_by("Bob", "Alice"), T)': {6},
 '-observed(greeted_by("Bob", "Alice"), T)': {1, 2, 3, 4, 5}}

In [6]:
action_string = "^(-?)(happened|observed)\(\s*(.*)\s*,\s*(T)\s*\)$"
action_pattern = re.compile(action_string)


def fill_time(fact, time):
    if action_pattern.match(fact):
        return action_pattern.sub(r"\1\2(\3, {})".format(time), fact)
    return fact



In [7]:
facts_set = set()

for fact in fact_dict:
    for time in fact_dict[fact]:
        facts_set.add(fill_time(fact, time))

facts_set

{'-observed(greeted_by("Bob", "Alice"), 1)',
 '-observed(greeted_by("Bob", "Alice"), 2)',
 '-observed(greeted_by("Bob", "Alice"), 3)',
 '-observed(greeted_by("Bob", "Alice"), 4)',
 '-observed(greeted_by("Bob", "Alice"), 5)',
 'happened(greet("Alice", "Bob"), 5)',
 'happened(move_through("Alice", "d12"), 2)',
 'happened(move_through("Alice", "d23"), 3)',
 'happened(move_through("Alice", "d34"), 4)',
 'happened(select(policy_compliant(greeted_by("Bob", "Alice"))), 0)',
 'happened(start(1), 1)',
 'happened(stop(1), 6)',
 'happened(wait, 0)',
 'happened(wait, 7)',
 'observed(greeted_by("Bob", "Alice"), 6)',
 'observed(in_room("Alice", "r2"), 3)',
 'observed(in_room("Alice", "r3"), 4)',
 'observed(in_room("Alice", "r4"), 5)'}

In [8]:
with open("run_observations.lp", "w") as lp_file:
    lp_file.writelines(["{}.\n".format(fact) for fact in facts_set])

In [9]:
apia_control_loop = os.path.join('..', 'Implementation', 'apia_control_loop.py')
instance = os.path.join('..', 'Examples', 'Authorization', 'Example_A', 'instance.lp')
domain_encoding = os.path.join('..', 'Examples', 'Authorization', 'Example_A', 'domain_encoding.lp')

In [10]:
subprocess.run(
    ["python3", apia_control_loop, '--max-timestep', '7', '--authorization-mode', 'paranoid', '--obligation-mode',
     'subordinate', instance, domain_encoding, "run_observations.lp"])

Running with configuration: APIAConfiguration(authorization=APIAAuthorizationSetting.PARANOID, obligation=APIAObligationSetting.SUBORDINATE)


Iteration 0
  Step 1: Interpret observations
    Grounding...
    Solving...
    Unobserved actions: 0

  Step 2: Find intended action
    Grounding...
    Solving...
    Intended action: wait

  Step 3: Do intended action
    Doing wait

  Step 4: Observe world
    Getting observations from #program observations_1.

Iteration 1
  Step 1: Interpret observations
    Grounding...
    Solving...
    Unobserved actions: 0

  Step 2: Find intended action
    Grounding...
    Solving...
    Intended action: start(1)
    New activity:
      activity_goal(2,policy_compliant(greeted_by("Bob","Alice")))

  Step 3: Do intended action
    Skipping start(1)

  Step 4: Observe world
    Getting observations from #program observations_2.

Iteration 2
  Step 1: Interpret observations
    Grounding...
    Solving...
    Unobserved actions: 0

  Step 2: Find intended action
    Grounding...
    Solving...
    Intended action: move_through("Alice","d12")

  Step 3: Do intended action
    Doing move_throu

CompletedProcess(args=['python3', '../Implementation/apia_control_loop.py', '--max-timestep', '7', '--authorization-mode', 'paranoid', '--obligation-mode', 'subordinate', '../Examples/Authorization/Example_A/instance.lp', '../Examples/Authorization/Example_A/domain_encoding.lp', 'run_observations.lp'], returncode=0)